In [32]:
import nltk
nltk.download("gutenberg")
from nltk.corpus import gutenberg
import pandas as pd
import numpy as np
data=gutenberg.raw('shakespeare-hamlet.txt')
with open("hamlet.txt","w") as fl:
    fl.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/workstation-p/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [34]:
with open("hamlet.txt","r") as fl:
    text=fl.read().lower()

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

4818

In [35]:
# create input sequences
input_sequences=[]
for line in text.split("\n"):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [36]:
max_sequence_len=max([len(x)for x in input_sequences])

In [37]:
max_sequence_len

14

In [38]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [39]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]],
      shape=(25732, 14), dtype=int32)

In [40]:
# create predictors and label
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [41]:
x,y

(array([[   0,    0,    0, ...,    0,    0,    1],
        [   0,    0,    0, ...,    0,    1,  687],
        [   0,    0,    0, ...,    1,  687,    4],
        ...,
        [   0,    0,    0, ...,  687,    4,   45],
        [   0,    0,    0, ...,    4,   45, 1047],
        [   0,    0,    0, ...,   45, 1047,    4]],
       shape=(25732, 13), dtype=int32),
 array([ 687,    4,   45, ..., 1047,    4,  193],
       shape=(25732,), dtype=int32))

In [42]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [43]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [44]:
# Train LSTM RNN
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Embedding(total_words,100))
model.add(tf.keras.layers.LSTM(150,return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
# tf.keras.layers.
model.add(tf.keras.layers.LSTM(50))
model.add(tf.keras.layers.Dense(total_words,activation='softmax'))

# Compile
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 13, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 50)             │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4818)           │       245,718 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 918,318 (3.50 MB)

 Trainable params: 918,318 (3.50 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [46]:
history=model.fit(X_train,y_train,epochs=500,validation_data=(X_test,y_test),verbose=1)

Epoch 1/500
604/604 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0323 - loss: 6.9365 - val_accuracy: 0.0340 - val_loss: 6.7221
Epoch 2/500
604/604 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0346 - loss: 6.4907 - val_accuracy: 0.0404 - val_loss: 6.8050
Epoch 3/500
604/604 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0455 - loss: 6.3578 - val_accuracy: 0.0480 - val_loss: 6.8384
Epoch 4/500
604/604 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0498 - loss: 6.2333 - val_accuracy: 0.0455 - val_loss: 6.8446
Epoch 5/500
604/604 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0516 - loss: 6.1424 - val_accuracy: 0.0466 - val_loss: 6.8800
Epoch 6/500
604/604 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0522 - loss: 6.0598 - val_accuracy: 0.0476 - val_loss: 6.9020
Epoch 7/500
604/604 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0568 - loss: 5.9644 - val_accuracy: 0.0538 - val_loss: 6.9377
Epoch 8/500
604/604 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0640 - loss: 5.8559 - val_accu

In [49]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list=token_list[-(max_sequence_len-1):]
    token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding="pre")
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None

In [50]:
input_text="To be or not to be"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"next word prediction:{next_word}")

Input text:To be or not to be
next word prediction:young


In [51]:
model.save("next_word_lstm.h5")

import pickle
with open("tokenizer.pkl","wb") as fl:
    pickle.dump(tokenizer,fl,protocol=pickle.HIGHEST_PROTOCOL)